<a href="https://colab.research.google.com/github/BatoolMM/PlantAI/blob/master/PlantAi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
!pip install -Uqq fastbook
!pip install fastcore==1.0.9
!pip install utils
fastbook.setup_book()
!pip install azure-cognitiveservices-search-imagesearch

     |████████████████████████████████| 727kB 4.4MB/s 
     |████████████████████████████████| 1.0MB 12.9MB/s 
     |████████████████████████████████| 358kB 23.6MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 51kB 8.0MB/s 
     |████████████████████████████████| 40kB 3.9MB/s 
     |████████████████████████████████| 40kB 6.9MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 61kB 9.2MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 2.6MB 23.0MB/s 
  Found existing installation: fastcore 1.0.13
    Uninstalling fastcore-1.0.13:
      Successfully uninstalled fastcore-1.0.13


NameError: ignored

In [ ]:
from fastbook import *
from fastai.vision.all import *
from fastai.vision.widgets import ImageClassifierCleaner
from utils import *
import os
from azure.cognitiveservices.search.imagesearch import ImageSearchClient as api
from msrest.authentication import CognitiveServicesCredentials as auth


In [ ]:
def search_images_bing(key, term, min_sz=128):
    client = api('https://api.cognitive.microsoft.com', auth(key))
    return L(client.images.search(query=term, count=150, min_height=min_sz, min_width=min_sz).value)
key = '8d86a0d22d7643549e560f18abfe8fde'
key2 ='d5851efce83544c6a29b96a7341f28dd'

In [ ]:
plants_type = 'Dieffenbachia', 'Schefflera', 'Dracaena', 'Epipremnum aureum'
path = Path('plants_new')

if not path.exists():
    path.mkdir()
    for o in plants_type:
      dest = (path/o)
      dest.mkdir(exist_ok =True)
      results = search_images_bing(key, f'{o} plant')
      download_images(dest, urls = results.attrgot('content_url'))

In [ ]:
files = get_image_files(path)
files

In [ ]:
failed = verify_images(files)
failed

In [ ]:
failed.map(Path.unlink)

In [ ]:
data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.1, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

In [ ]:
data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
for idx in cleaner.delete():
  cleaner.fns[idx].unlink()

In [ ]:
dls = data.dataloaders(path)
learn = cnn_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(7)

In [ ]:
interpret = ClassificationInterpretation.from_learner(learn)
interpret.plot_confusion_matrix()

In [ ]:
interpret.plot_top_losses(2,nrows=2)

In [ ]:
learn.export()